## Backtest Pipeline Example
In this example, we will run a full backtest pipeline for a trading strategy.

### Steps:

1. Download a insample dataset to detect backtest date ranges
2. Run strategy over each range and save results
3. Show results in a backtest report

## Setup constants for backtesting

In [1]:
from datetime import datetime, timezone
from investing_algorithm_framework import BacktestDateRange

# Constants
STRATEGY_NAME = 'StrategyV1'
DATA_STORAGE_PATH = "./resources/backtest_data"
BACKTEST_RESULTS_STORAGE_PATH = "./resources/backtest_results"
MARKET = "BITVAVO"
RISK_FREE_RATE = 0.027  # Example risk-free rate, adjust as needed
WINDOW_SIZE = 200  # Example window size, adjust as needed

# Asset constants
ASSETS = ["BTC/EUR"]

# Time range constants
insample_date_range = BacktestDateRange(
    start_date=datetime(2022, 1, 1, tzinfo=timezone.utc),
    end_date=datetime(2024, 12, 31, tzinfo=timezone.utc),
)

strategy_params = {
    'ema_short_period': 50,
    'ema_long_period': 200,
    'rsi_period': 14,
    'rsi_overbought_threshold': 70,
    'rsi_oversold_threshold': 30,
    'alignment_window_size':48,
}

## Dynamically loadin the strategy
We can dynamically load the strategy from a file. This is useful if we want to change the strategy without restarting the entire
notebook. This allows you to edit the strategy and quickly test it.

In [2]:
import importlib
import strategies.strategy_v1.strategy_v1 as ema_crossover_v1

importlib.reload(ema_crossover_v1)
EMACrossoverRSIStrategy = ema_crossover_v1.EMACrossoverRSIStrategy

## Select in sample test date ranges

Date ranges for in-sample testing should be atleast 2 years long so we can calculate 
all metrics. Three date ranges are selected, coresponding to an uptrend, a downtrend and a sideways trend. Using these three date ranges we can test the performance of the strategy in different market conditions.

We will use a window=730, which means 2-year slices of market behavior that we can use to detect uptrends, downtrends and sideways trends.


In [3]:
from investing_algorithm_framework import download

insample_data = download(
    symbol="BTC/EUR",
    start_date=insample_date_range.start_date,
    end_date=insample_date_range.end_date,
    market=MARKET,
    time_frame="1d",
    save=True,
    storage_path=DATA_STORAGE_PATH
)

In [4]:
from investing_algorithm_framework import select_backtest_date_ranges

# Select date ranges for backtesting with a window of 730 days (2 years), so
# we have clear uptrend, downtrend, and sideways market conditions.
ranges = select_backtest_date_ranges(insample_data, window=730)

for r in ranges:
    print(r)

UpTurn: 2022-12-18 00:00:00+00:00 - 2024-12-17 00:00:00+00:00
DownTurn: 2022-01-01 00:00:00+00:00 - 2024-01-01 00:00:00+00:00
SideWays: 2022-01-05 00:00:00+00:00 - 2024-01-05 00:00:00+00:00


## Visualize the strategy indicators

In [5]:
import pandas as pd
import plotly.graph_objects as go


def create_crossover_chart(
    data: pd.DataFrame,
    column="crossover",
    color='green'
):
    crossover = data[data[column] == True]
    plot = go.Scatter(
        x=crossover.index,
        y=crossover['Close'],
        mode='markers',
        name='Crossover',
        marker=dict(color=color, size=10, symbol='cross')
    )
    return plot

def create_crossunder_chart(
    data: pd.DataFrame,
    column="crossunder",
    color='red'
):
    crossunder = data[data[column] == True]
    plot = go.Scatter(
        x=crossunder.index,
        y=crossunder['Close'],
        mode='markers',
        name='Crossunder',
        marker=dict(color=color, size=10, symbol='cross')
    )
    return plot

def create_line_chart(
    data: pd.DataFrame,
    column="Close",
    color='blue',
):
    plot = go.Scatter(
        x=data.index,
        y=data[column],
        mode='lines',
        name='Close',
        line=dict(color=color)
    )
    return plot

In [6]:
from plotly.subplots import make_subplots
from pyindicators import ema, rsi, crossover, crossunder

data = insample_data.copy()
data = ema(
    data=data,
    period=strategy_params['ema_short_period'],
    source_column='Close',
    result_column='ema_short'
)
data = ema(
    data=data,
    period=strategy_params['ema_long_period'],
    source_column='Close',
    result_column='ema_long'
)
data = rsi(
    data=data,
    period=strategy_params['rsi_period'],
    source_column='Close',
    result_column='rsi'
)

data = crossover(
    data=data,
    first_column='ema_short',
    second_column='ema_long',
    result_column='crossover'
)
data = crossunder(
    data=data,
    first_column='ema_short',
    second_column='ema_long',
    result_column='crossunder'
)
fig = make_subplots(
    rows=2,
    cols=1,
    shared_xaxes=True,
    vertical_spacing=0.02,
    subplot_titles=("EMA Crossovers", "RSI"),
    row_heights=[0.4, 0.2]
)
fig.add_trace(
    create_crossover_chart(data, column="crossover", color='green'),
    row=1,
    col=1
)
fig.add_trace(
    create_crossunder_chart(data, column="crossunder", color='red'),
    row=1,
    col=1
)
fig.add_trace(
    create_line_chart(data, column="rsi", color='purple'),
    row=2,
    col=1
)
fig.add_trace(
    create_line_chart(data, column="ema_long", color='orange'),
    row=1,
    col=1
)
fig.add_trace(
    create_line_chart(data, column="ema_short", color='green'),
    row=1,
    col=1
)
fig.add_trace(
    create_line_chart(data, column="Close", color='blue'),
    row=1,
    col=1
)
fig.update_layout(
    height=800,
    title_text="EMA Crossover Strategy Visualization",
    showlegend=False,
    margin=dict(l=50, r=50, t=50, b=50),
)

fig.show()

## Run in-sample backtest
We will run the strategy over each date range and save the results. This allows us to analyze the performance of the strategy in different market conditions. We will save the results to a file so we can analyze them later

In [7]:
from investing_algorithm_framework import create_app, DataSource, DataType, \
    PortfolioConfiguration, RESOURCE_DIRECTORY, SnapshotInterval

app = create_app(
    config={
        RESOURCE_DIRECTORY: "./resources",
    }
)
app.add_portfolio_configuration(
    PortfolioConfiguration(
        initial_balance=1000,
        trading_symbol="EUR",
        market=MARKET,
    )
)
data_sources = []

for asset in ASSETS:
    data_sources.append(
        DataSource(
            identifier=f"OHLCV_{asset}",
            data_type=DataType.OHLCV,
            time_frame="4h",
            market=MARKET,
            symbol=asset,
            window_size=WINDOW_SIZE
        )
    )

strategy = EMACrossoverRSIStrategy(
    time_unit="hour",
    interval=2,
    symbols=["BTC/EUR"],
    data_sources=data_sources,
    market=MARKET,
    ema_short_period=strategy_params["ema_short_period"],
    ema_long_period=strategy_params["ema_long_period"],
    rsi_period=strategy_params["rsi_period"],
    rsi_oversold_threshold=strategy_params["rsi_oversold_threshold"],
    rsi_overbought_threshold=strategy_params["rsi_overbought_threshold"],
    alignment_window_size=strategy_params["alignment_window_size"],
)
backtests = []
backtest_date_range = ranges[0]  # Example: using the first date range for backtesting
# for backtest_date_range in ranges:
backtest = app.run_backtest(
    strategy=strategy,
    backtest_date_range=backtest_date_range,
    initial_amount=1000,
    snapshot_interval=SnapshotInterval.DAILY,
    risk_free_rate=RISK_FREE_RATE,
    save=False
)
backtests.append(backtest)


Preparing backtest data for all data sources:   0%|          | 0/1 [00:00<?, ?it/s]

Running backtest:   0%|          | 0/8761 [00:00<?, ?it/s]

In [9]:
for backtest in backtests:
    backtest.save(
        directory_path=BACKTEST_RESULTS_STORAGE_PATH,
    )

## Show in sample backtest results


In [11]:
from investing_algorithm_framework import BacktestReport, Backtest

backtest = Backtest.open(
    directory_path=BACKTEST_RESULTS_STORAGE_PATH
)
report = BacktestReport.open(
    backtests=[backtest]
)
report.show(browser=True)

Start date must be a UTC datetime object. Received: 2022-12-18 00:00:00
End date must be a UTC datetime object. Received: 2024-12-17 00:00:00


Metric,Value
Start Date,2022-12-17 23:00:00+00:00
End Date,2024-12-16 23:00:00+00:00
% Winning Months,50.00%
% Winning Years,100.00%
AVG Mo Return,3.41%
AVG Mo Return (Losing Months),-2.97%
AVG Mo Return (Winning Months),8.30%
Best Month,24.71% 2024-02-29 00:00:00+00:00
Worst Month,-5.78% 2024-06-30 00:00:00+00:00
Best Year,67.57% 2024-01-01
